In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
import random

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the Hotdog_NotHotdog dataset class
class Hotdog_NotHotdog(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path='C:/Users/nicla/DTU/ComputerVision/1st Poster/hotdog_nothotdog'):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path + '/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: id for id, c in enumerate(image_classes)}
        self.image_paths = glob.glob(data_path + '/*/*.jpg')
        
    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        c = os.path.split(os.path.split(image_path)[0])[1]
        y = self.name_to_label[c]
        X = self.transform(image)
        return X, y

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 64 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

# Early stopping utility
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0


# Function to calculate accuracy
def calculate_accuracy(loader, model, device):
    model.eval()  
    correct = 0
    total = 0
    
    with torch.no_grad(): 
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float() 
            correct += (predicted.squeeze() == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

# Training function with early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device):
    model.to(device)  # Move model to the specified device
    train_accuracies = []
    val_accuracies = []
    
    early_stopping = EarlyStopping(patience=patience)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for i, data in enumerate(progress_bar):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            progress_bar.set_postfix({'Loss': running_loss / (i + 1)})

        # Calculate accuracy on training and validation datasets
        train_acc = calculate_accuracy(train_loader, model, device)
        val_acc = calculate_accuracy(val_loader, model, device)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, '
              f'Train Accuracy: {train_acc:.2f}%, Validation Accuracy: {val_acc:.2f}%')

        # Early stopping check
        early_stopping(val_acc)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    return train_accuracies, val_accuracies

# Main
def run_experiment(augmentation_methods, num_runs=5):
    size = 128
    batch_size = 64
    num_epochs = 50
    patience = 10
    results = []

    base_transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor()
    ])

    # Define augmentation methods
    color_jitter = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    random_rotation = transforms.RandomRotation(degrees=15)
    random_horizontal_flip = transforms.RandomHorizontalFlip(p=0.5)
    random_affine = transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2))
    random_resized_crop = transforms.RandomResizedCrop(size, scale=(0.8, 1.0))

    augmentation_dict = {
        'color_jitter': color_jitter,
        'random_rotation': random_rotation,
        'random_horizontal_flip': random_horizontal_flip,
        'random_affine': random_affine,
        'random_resized_crop': random_resized_crop
    }

    # Create all combinations of augmentation methods
    augmentation_combinations = [
        [],
        ['color_jitter'],
        ['random_rotation'],
        ['random_horizontal_flip'],
        ['random_affine'],
        ['random_resized_crop'],
        ['color_jitter', 'random_rotation', 'random_horizontal_flip', 'random_affine', 'random_resized_crop']
    ]

    for combo in augmentation_combinations:
        combo_name = '+'.join(combo) if combo else 'no_augmentation'
        print(f"\nRunning experiment with {combo_name}")

        for run in range(num_runs):
            print(f"\nRun {run + 1}/{num_runs}")

            # Create the transform for this combination
            train_transform = transforms.Compose([
                transforms.Resize((size, size)),
                *[augmentation_dict[aug] for aug in combo],
                transforms.ToTensor()
            ])

            # Load and split the datasets
            trainset = Hotdog_NotHotdog(train=True, transform=train_transform)
            train_size = int(0.8 * len(trainset))
            val_size = len(trainset) - train_size
            train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

            testset = Hotdog_NotHotdog(train=False, transform=base_transform)
            test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

            # Initialize the model, loss function, and optimizer
            model = SimpleCNN()
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=0.0002)

            # Train the model with early stopping
            train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device)

            # Calculate test accuracy
            test_accuracy = calculate_accuracy(test_loader, model, device)

            # Save results
            results.append({
                'augmentation': combo_name,
                'run': run + 1,
                'train_accuracy': train_accuracies[-1],
                'val_accuracy': val_accuracies[-1],
                'test_accuracy': test_accuracy
            })

    # Save results to CSV
    with open('hotdog_classification_results.csv', 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['augmentation', 'run', 'train_accuracy', 'val_accuracy', 'test_accuracy'])
        writer.writeheader()
        for result in results:
            writer.writerow(result)

    print("\nExperiment completed. Results saved to 'hotdog_classification_results.csv'")


# Run the experiment
if __name__ == "__main__":
    run_experiment(['color_jitter', 'random_rotation', 'random_horizontal_flip'])



Running experiment with no_augmentation

Run 1/5


Epoch 1/50: 100%|██████████████████████████████████████████████████████████| 26/26 [00:03<00:00,  8.23it/s, Loss=0.669]


Epoch [1/50], Loss: 0.6692, Train Accuracy: 53.27%, Validation Accuracy: 53.17%


Epoch 2/50: 100%|██████████████████████████████████████████████████████████| 26/26 [00:03<00:00,  8.14it/s, Loss=0.529]


Epoch [2/50], Loss: 0.5289, Train Accuracy: 78.62%, Validation Accuracy: 76.10%


...

Epoch [9/50], Loss: 0.5024, Train Accuracy: 78.07%, Validation Accuracy: 77.32%


Epoch 10/50: 100%|█████████████████████████████████████████████████████████| 26/26 [00:06<00:00,  3.86it/s, Loss=0.488]


Epoch [10/50], Loss: 0.4883, Train Accuracy: 77.83%, Validation Accuracy: 79.51%


Epoch 11/50: 100%|█████████████████████████████████████████████████████████| 26/26 [00:06<00:00,  3.79it/s, Loss=0.481]


Epoch [11/50], Loss: 0.4813, Train Accuracy: 78.86%, Validation Accuracy: 81.95%
Early stopping

Experiment completed. Results saved to 'hotdog_classification_results.csv'
